In [1]:
import pandas as pd
import geopandas as gpd
import rioxarray
import math
import shapely
import xvec

from shapely.geometry import Point, MultiPoint

In [2]:
sightlines = pd.read_pickle("my_results/method_1/d06_sightlines_geometries.pickle")
sightlines = sightlines[["start_point", "end_point", "sight_line_points"]]

# Convert tuples with coordinates to shapely geometry
Should eventually be deprecated by already using shapely geometries
in the first place.

In [3]:
def convert_to_point(row):
    return Point(row)

In [4]:
def convert_to_multipoint(row):
    if row:
        return MultiPoint(row)
    else:
        return None

In [5]:
# Convert start and end point to geometry
tuple_columns = ["start_point", "end_point"]

for col in tuple_columns:
    sightlines[col+"_geom"] = sightlines[col].apply(convert_to_point)


# Convert sight line points to geometry
sightlines["sl_points"] = sightlines["sight_line_points"].apply(convert_to_multipoint)

# Load raster dtm

In [6]:
raster = rioxarray.open_rasterio("data/dtm/dtm_raw/RGEALTI_FXX_1025_6285_MNT_LAMB93_IGN69.asc")
raster = raster.drop_vars("band").squeeze()

# Extract z coordinates from raster

In [7]:
# Extract z coords from raster
z_start = raster.drop_vars("spatial_ref").xvec.extract_points(points=sightlines["start_point_geom"], x_coords="x",y_coords="y")

# Extract z coords from raster
z_end = raster.drop_vars("spatial_ref").xvec.extract_points(points=sightlines["end_point_geom"], x_coords="x",y_coords="y")

In [ ]:
# TODO
# This is waayy to complicated. Should be reworked

new_gdf = gpd.GeoDataFrame(columns=['geometry'], geometry='geometry')

for row in sightlines["sight_line_points_geom"]:
    if row is not None:
        points = row.geoms
        
        z_points = raster.drop_vars("spatial_ref").xvec.extract_points(points=points, x_coords="x",y_coords="y")
        z_points = z_points.xvec.to_geopandas().rename(columns={0: "z"})

        z_points["geometry"] = z_points.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
        z_points = z_points.drop(columns="z")
        
        multipoint = MultiPoint(z_points["geometry"].tolist())
        
        row_gdf = gpd.GeoDataFrame({'geometry': [multipoint]}, geometry='geometry')

        new_gdf = pd.concat([new_gdf, row_gdf], ignore_index=True)
    else:
        new_row = gpd.GeoDataFrame({'geometry': [None]}, geometry='geometry')
        new_gdf = pd.concat([new_gdf, new_row], ignore_index=True)

## Put results together

In [ ]:
# Convert do gpd
z_start = z_start.xvec.to_geopandas().rename(columns={0: "z"})

# Append z values to points
z_start["start_point_3d"] = z_start.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
z_start = z_start.drop(columns=["geometry", "z"])

# Convert do gpd
z_end = z_end.xvec.to_geopandas().rename(columns={0: "z"})

# Append z values to points
z_end["end_point_3d"] = z_end.apply(lambda row: Point(row["geometry"].x, row["geometry"].y, row["z"]), axis=1)
z_end = z_end.drop(columns=["geometry", "z"])

In [ ]:
sightlines = pd.concat([z_start, z_end, new_gdf], axis=1)

sightlines = sightlines.rename(columns={"start_point_3d": "sl_start",
                                          "end_point_3d": "sl_end",
                                          "geometry": "sl_points"})

In [ ]:
sightlines

# Compute slope function
Finally, yaaay.

In [ ]:
def compute_slope(road_row):
    
    start = road_row.sl_start
    end = road_row.sl_end
    slp = road_row.sl_points

    if slp == None:
        slope_percent = abs(start.z - end.z) / shapely.distance(start, end)
        slope_degree = math.degrees(math.atan(slope_percent))

        return slope_percent, slope_degree
    
    slp_list = [p for p in slp.geoms]

    points = []

    points.append(start)
    for p in slp_list:
        points.append(p)
    points.append(end)

    # number of points
    nb_points = len([start]) + len([end]) + len(slp_list)
    sum_slope_percent = 0
    sum_slope_radian = 0
    sum_nb_points = 0
    # if theres one or more sl_point
    for i in range (1, nb_points-1):
        a = points[i-1]
        b = points[i+1]
            
        sum_nb_points += 1
        inter_slope_percent = abs(a.z - b.z) / shapely.distance(a, b)

        sum_slope_percent += inter_slope_percent
        sum_slope_radian += math.atan(inter_slope_percent)
        
    slope_percent = sum_slope_percent/sum_nb_points
    slope_degree = math.degrees(sum_slope_radian/sum_nb_points)

    return slope_degree, slope_percent

# Compute results

In [ ]:
slope_values = []

for _, road_row in sightlines.iterrows():

    # display([p for p in road_row.sl_points.geoms])
    # break

    slope_degree, slope_percent = compute_slope(road_row)

    slope_values.append([slope_degree, slope_percent])

    df_slopes = pd.DataFrame(slope_values,
                             columns=["slope_degree",
                                      "slope_percent"])

In [ ]:
df_slopes